# Modifique los códigos de Runge-Kutta-Fehlberg y Adams con control de error para que impriman el número de pasos requeridos. Luego resuelva el siguiente problema de condición incial con ambos métodos y compare para diferentes tolerancias:

$$
y'(t) = (1/t)(y^2 + y)\,,\quad 1 \leq t \leq 3\,,\quad y(1) = -2\,.
$$

##Método de Runge-Kutta-Fehlberg (RKF45) con control de error y contador de pasos:

In [2]:
def rkf45(f, a, b, y0, tol):
    t = a
    y = y0
    h = 0.1  # Tamaño inicial de paso
    steps = 0  # Inicializar contador de pasos

    while t < b:
        k1 = h * f(t, y)
        k2 = h * f(t + h / 4, y + k1 / 4)
        k3 = h * f(t + 3 * h / 8, y + 3 / 32 * k1 + 9 / 32 * k2)
        k4 = h * f(t + 12 * h / 13, y + 1932 / 2197 * k1 - 7200 / 2197 * k2 + 7296 / 2197 * k3)
        k5 = h * f(t + h, y + 439 / 216 * k1 - 8 * k2 + 3680 / 513 * k3 - 845 / 4104 * k4)
        k6 = h * f(t + h / 2, y - 8 / 27 * k1 + 2 * k2 - 3544 / 2565 * k3 + 1859 / 4104 * k4 - 11 / 40 * k5)

        R = 1 / h * abs(1 / 360 * k1 - 128 / 4275 * k3 - 2197 / 75240 * k4 + 1 / 50 * k5 + 2 / 55 * k6)
        delta = 0.84 * (tol / R) ** 0.25

        if R <= tol:
            t += h
            y += 25 / 216 * k1 + 1408 / 2565 * k3 + 2197 / 4104 * k4 - 1 / 5 * k5
            steps += 1

        h = min(2 * h, delta * h) if delta >= 1 else delta * h

    return y, steps


##Método de Adams con control de error y contador de pasos:

In [3]:
def adams(f, a, b, y0, tol):
    t = a
    y = y0
    h = 0.1  # Tamaño inicial de paso
    steps = 0  # Inicializar contador de pasos
    prev_vals = []

    while t < b:
        if len(prev_vals) < 4:
            k1 = h * f(t, y)
            k2 = h * f(t + h / 2, y + k1 / 2)
            k3 = h * f(t + h / 2, y + k2 / 2)
            k4 = h * f(t + h, y + k3)

            y_next = y + 1 / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
            prev_vals.append((t, y))
            t += h
            y = y_next
            steps += 1

        else:
            predictor = y + h / 24 * (55 * f(t, y) - 59 * f(prev_vals[-1][0], prev_vals[-1][1]) +
                                      37 * f(prev_vals[-2][0], prev_vals[-2][1]) - 9 * f(prev_vals[-3][0], prev_vals[-3][1]))

            corrector = predictor
            error = tol * 2  # Inicializar error con un valor mayor que la tolerancia
            while error > tol:
                k1 = h * f(t, corrector)
                k2 = h * f(t + h / 2, corrector + k1 / 2)
                k3 = h * f(t + h / 2, corrector + k2 / 2)
                k4 = h * f(t + h, corrector + k3)

                y_next = corrector + 1 / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
                error = abs(y_next - corrector)
                corrector = y_next
                steps += 1

            t += h
            y = corrector
            prev_vals.pop(0)
            prev_vals.append((t, y))

    return y, steps


In [4]:
import numpy as np

# Definir la ecuación diferencial y sus parámetros
def f(t, y):
    return (1 / t) * (y ** 2 + y)

a = 1
b = 3
y0 = -2

# Definir las tolerancias para la comparación
tolerances = [1e-6, 1e-8, 1e-10]

for tol in tolerances:
    print(f"\nTolerancia: {tol}")

    # Método de Runge-Kutta-Fehlberg (RKF45)
    result_rkf, steps_rkf = rkf45(f, a, b, y0, tol)
    print(f"RKF45: Resultado = {result_rkf}, Pasos = {steps_rkf}")

    # Método de Adams
    result_adams, steps_adams = adams(f, a, b, y0, tol)
    print(f"Adams: Resultado = {result_adams}, Pasos = {steps_adams}")



Tolerancia: 1e-06
RKF45: Resultado = -1.1592646633076549, Pasos = 13
Adams: Resultado = -1.00000519307231, Pasos = 423

Tolerancia: 1e-08
RKF45: Resultado = -1.194055757817298, Pasos = 34
Adams: Resultado = -1.000000051766889, Pasos = 637

Tolerancia: 1e-10
RKF45: Resultado = -1.1984947169921412, Pasos = 101
Adams: Resultado = -1.000000000517139, Pasos = 851
